In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from keras.layers import BatchNormalization, Lambda, Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.layers.merge import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = 'ignore', category= FutureWarning)
%matplotlib inline


In [14]:
def mycolournet(num_classes):
    input_image = Input(shape=(56,56,3))
    layer1 = MaxPooling2D(pool_size=(1,1),strides=(1,1),input_shape=(56,56,3))(input_image)
    
    blueinput =  Lambda(lambda x : x[:,:,:,0:1])(layer1)
    greeninput =  Lambda(lambda x : x[:,:,:,1:2])(layer1)
    redinput =  Lambda(lambda x : x[:,:,:,:2])(layer1)
    
    
    
    blueconv1 = Convolution2D(filters=16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(blueinput)
    blueconv1 = BatchNormalization()(blueconv1)
    blueconv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(blueconv1)
    
    greenconv1 = Convolution2D(filters=16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(greeninput)
    greenconv1 = BatchNormalization()(greenconv1)
    greenconv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(greenconv1)
    
    redconv1 = Convolution2D(filters=16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(redinput)
    redconv1 = BatchNormalization()(redconv1)
    redconv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(redconv1)
    
    blueconv2 = Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(blueconv1)
    blueconv2 = BatchNormalization()(blueconv2)
    blueconv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(blueconv2)
    
    greenconv2 = Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(greenconv1)
    greenconv2 = BatchNormalization()(greenconv2)
    greenconv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(greenconv2)
    
    redconv2 = Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(redconv1)
    redconv2 = BatchNormalization()(redconv2)
    redconv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(redconv2)
    
    blueflatten = Flatten()(blueconv2)
    greenflatten = Flatten()(greenconv2)
    redflatten = Flatten()(redconv2)
    
    blueFC_1 = Dense(units=200, activation='relu')(blueflatten)
    blueFC_1 = Dropout(0.6)(blueFC_1)
    blueFC_2 = Dense(units=1, activation='relu')(blueFC_1)
    
    greenFC_1 = Dense(units=200, activation='relu')(greenflatten)
    greenFC_1 = Dropout(0.6)(greenFC_1)
    greenFC_2 = Dense(units=1, activation='relu')(greenFC_1)
    
    redFC_1 = Dense(units=200, activation='relu')(redflatten)
    redFC_1 = Dropout(0.6)(redFC_1)
    redFC_2 = Dense(units=1, activation='relu')(redFC_1)
    
    concatoutput = Concatenate()([blueFC_2, greenFC_2,redFC_2])
    
    commonFC_1 = Dense(units=20, activation='relu'  )(concatoutput)
    commonFC_2 = Dense(units=40, activation = 'relu')(commonFC_1)
    commonFC_3 = Dense(units=20, activation= 'relu')(commonFC_2)
    output = Dense(units=num_classes,activation='softmax' )(commonFC_3)
    
    model = Model(inputs=input_image,outputs=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    # sgd = SGD(lr=0.01, momentum=0.9, decay=0.0005, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [15]:
model = mycolournet(10)

In [16]:
train_path = 'data/colors/train'
valid_path = 'data/colors/valid'
test_path = 'data/colors/test'

In [17]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10, shuffle=False)

Found 1301 images belonging to 10 classes.
Found 83 images belonging to 10 classes.
Found 2 images belonging to 10 classes.


In [19]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=500,
    verbose=2)

Epoch 1/500
131/131 - 36s - loss: 0.3014 - accuracy: 0.8947 - val_loss: 2.0723 - val_accuracy: 0.4217
Epoch 2/500
131/131 - 15s - loss: 0.2681 - accuracy: 0.9085 - val_loss: 2.3123 - val_accuracy: 0.3855
Epoch 3/500
131/131 - 15s - loss: 0.2397 - accuracy: 0.9216 - val_loss: 2.3141 - val_accuracy: 0.3976
Epoch 4/500
131/131 - 16s - loss: 0.2100 - accuracy: 0.9362 - val_loss: 2.2420 - val_accuracy: 0.3735
Epoch 5/500
131/131 - 16s - loss: 0.2373 - accuracy: 0.9247 - val_loss: 2.4107 - val_accuracy: 0.4096
Epoch 6/500
131/131 - 16s - loss: 0.2769 - accuracy: 0.9008 - val_loss: 2.3825 - val_accuracy: 0.4458
Epoch 7/500
131/131 - 17s - loss: 0.2661 - accuracy: 0.9116 - val_loss: 2.6917 - val_accuracy: 0.3494
Epoch 8/500
131/131 - 17s - loss: 0.2442 - accuracy: 0.9216 - val_loss: 2.5773 - val_accuracy: 0.3855
Epoch 9/500
131/131 - 16s - loss: 0.2526 - accuracy: 0.9131 - val_loss: 2.3335 - val_accuracy: 0.4096
Epoch 10/500
131/131 - 16s - loss: 0.2818 - accuracy: 0.9047 - val_loss: 2.8222 - 

Epoch 81/500
131/131 - 17s - loss: 0.1437 - accuracy: 0.9577 - val_loss: 2.3721 - val_accuracy: 0.4096
Epoch 82/500
131/131 - 17s - loss: 0.1313 - accuracy: 0.9570 - val_loss: 2.3967 - val_accuracy: 0.4096
Epoch 83/500
131/131 - 18s - loss: 0.1945 - accuracy: 0.9400 - val_loss: 2.5133 - val_accuracy: 0.4458
Epoch 84/500
131/131 - 18s - loss: 0.1381 - accuracy: 0.9554 - val_loss: 2.6391 - val_accuracy: 0.4217
Epoch 85/500
131/131 - 18s - loss: 0.1622 - accuracy: 0.9500 - val_loss: 2.8804 - val_accuracy: 0.3735
Epoch 86/500
131/131 - 17s - loss: 0.2269 - accuracy: 0.9354 - val_loss: 2.1942 - val_accuracy: 0.4096
Epoch 87/500
131/131 - 38s - loss: 0.1734 - accuracy: 0.9454 - val_loss: 2.5310 - val_accuracy: 0.4337
Epoch 88/500
131/131 - 17s - loss: 0.1756 - accuracy: 0.9470 - val_loss: 2.4145 - val_accuracy: 0.4578
Epoch 89/500
131/131 - 17s - loss: 0.1919 - accuracy: 0.9424 - val_loss: 2.6028 - val_accuracy: 0.3855
Epoch 90/500
131/131 - 17s - loss: 0.1468 - accuracy: 0.9608 - val_loss: 

Epoch 160/500
131/131 - 19s - loss: 0.1397 - accuracy: 0.9623 - val_loss: 3.5602 - val_accuracy: 0.3614
Epoch 161/500
131/131 - 18s - loss: 0.3605 - accuracy: 0.8901 - val_loss: 2.9425 - val_accuracy: 0.3373
Epoch 162/500
131/131 - 18s - loss: 0.1748 - accuracy: 0.9447 - val_loss: 3.0367 - val_accuracy: 0.3614
Epoch 163/500
131/131 - 18s - loss: 0.1458 - accuracy: 0.9554 - val_loss: 2.9151 - val_accuracy: 0.4217
Epoch 164/500
131/131 - 18s - loss: 0.1166 - accuracy: 0.9639 - val_loss: 2.9416 - val_accuracy: 0.3855
Epoch 165/500
131/131 - 18s - loss: 0.1253 - accuracy: 0.9639 - val_loss: 2.8319 - val_accuracy: 0.4217
Epoch 166/500
131/131 - 18s - loss: 0.1336 - accuracy: 0.9623 - val_loss: 3.0412 - val_accuracy: 0.3855
Epoch 167/500
131/131 - 18s - loss: 0.1123 - accuracy: 0.9700 - val_loss: 3.3581 - val_accuracy: 0.3976
Epoch 168/500
131/131 - 18s - loss: 0.1362 - accuracy: 0.9608 - val_loss: 3.2177 - val_accuracy: 0.3855
Epoch 169/500
131/131 - 18s - loss: 0.6035 - accuracy: 0.8040 - 

KeyboardInterrupt: 

In [18]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=150,
    verbose=2)

Epoch 1/150
131/131 - 18s - loss: 2.2733 - accuracy: 0.1491 - val_loss: 2.0829 - val_accuracy: 0.2169
Epoch 2/150
131/131 - 18s - loss: 2.1361 - accuracy: 0.2490 - val_loss: 2.0713 - val_accuracy: 0.2169
Epoch 3/150
131/131 - 18s - loss: 2.0331 - accuracy: 0.2437 - val_loss: 2.0341 - val_accuracy: 0.2169
Epoch 4/150
131/131 - 18s - loss: 1.9972 - accuracy: 0.2529 - val_loss: 1.9108 - val_accuracy: 0.2169
Epoch 5/150
131/131 - 18s - loss: 1.9692 - accuracy: 0.2414 - val_loss: 1.9547 - val_accuracy: 0.2169
Epoch 6/150
131/131 - 18s - loss: 1.9330 - accuracy: 0.2613 - val_loss: 1.9021 - val_accuracy: 0.2289
Epoch 7/150
131/131 - 18s - loss: 1.8858 - accuracy: 0.2613 - val_loss: 1.9229 - val_accuracy: 0.2289
Epoch 8/150
131/131 - 17s - loss: 1.8511 - accuracy: 0.2675 - val_loss: 1.8326 - val_accuracy: 0.2410
Epoch 9/150
131/131 - 17s - loss: 1.7993 - accuracy: 0.2829 - val_loss: 1.7744 - val_accuracy: 0.2530
Epoch 10/150
131/131 - 18s - loss: 1.7985 - accuracy: 0.2752 - val_loss: 1.8311 - 

Epoch 81/150
131/131 - 18s - loss: 0.5858 - accuracy: 0.7848 - val_loss: 1.5591 - val_accuracy: 0.3735
Epoch 82/150
131/131 - 18s - loss: 0.5093 - accuracy: 0.8178 - val_loss: 1.5688 - val_accuracy: 0.4217
Epoch 83/150
131/131 - 18s - loss: 0.5703 - accuracy: 0.7733 - val_loss: 1.6519 - val_accuracy: 0.3976
Epoch 84/150
131/131 - 18s - loss: 0.5499 - accuracy: 0.7909 - val_loss: 1.5649 - val_accuracy: 0.3855
Epoch 85/150
131/131 - 17s - loss: 0.5170 - accuracy: 0.7978 - val_loss: 1.7598 - val_accuracy: 0.4096
Epoch 86/150
131/131 - 17s - loss: 0.5049 - accuracy: 0.8078 - val_loss: 1.6566 - val_accuracy: 0.4458
Epoch 87/150
131/131 - 18s - loss: 0.4831 - accuracy: 0.8209 - val_loss: 1.7228 - val_accuracy: 0.4699
Epoch 88/150
131/131 - 18s - loss: 0.5258 - accuracy: 0.7940 - val_loss: 1.6296 - val_accuracy: 0.4699
Epoch 89/150
131/131 - 18s - loss: 0.5506 - accuracy: 0.7971 - val_loss: 1.7292 - val_accuracy: 0.4337
Epoch 90/150
131/131 - 18s - loss: 0.4745 - accuracy: 0.8248 - val_loss: 